# Create a Prescription Parser using CRF

## Install necessary libraries

### Install nltk, sklearn and sklearn_crfsuite (or python_crfsuite) using pip install

## Import the libraries

In [1]:
from nltk.tag import pos_tag
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
import pycrfsuite

## Create training data with labels

#### Creating the inputs to the ML model in the following form: ((GIVEN as INPUT))
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [2]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [3]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

#### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [4]:
def tuples_maker(inp, out):
    sample_data = []
    for i in range(len(inp)):
        sentence_tuples = []
        for j in range(len(inp[i])):
            input_word = inp[i][j]
            output_word = out[i][j]
            word_tuple = (input_word, output_word)
            sentence_tuples.append(word_tuple)
        sample_data.append(sentence_tuples)
    return sample_data

In [5]:
whole_data = tuples_maker(input_sigs, output_labels)
whole_data

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')],
 [('inject', 'Method'), ('2', 'Qty'), ('units', 'Form')],
 [('x', 'FOR'), ('2', 'Duration'), ('weeks', 'DurationUnit')],
 [('x', 'FOR'), ('3', 'Duration'), ('days', 'DurationUnit')],
 [('every', 'EVERY'), ('day', 'Period')],
 [('every', 'EVERY'), ('2', 'Period'), ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'), ('3', 'Period'), ('days', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('1', 'Period'),
  ('to', 'TO'),
  ('2', 'PeriodMax'),
  ('months', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('2', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('weeks', 'PeriodUnit')],
 [('every', 'EVERY'),
  ('4', 'Period'),
  ('to', 'TO'),
  ('6', 'PeriodMax'),
  ('days', 'PeriodUnit')],
 [('take', 'Method'),
  ('two', 'Qty'),
  ('to', 'TO'),
  ('four', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('2', 'Qty'),
  ('to', 'TO'),
  ('4', 'Qty'),
  ('tabs', 'Form')],
 [('take', 'Method'),
  ('3', 

#### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [6]:
def triples_maker(whole_data):
    sample_data = []
    for sentence in whole_data:
        tags = pos_tag([word_tuple[0] for word_tuple in sentence])
        triples = [(word_tuple[0], tag[1], word_tuple[1]) for word_tuple, tag in zip(sentence, tags)]
        sample_data.append(triples)
    return sample_data

In [7]:
sample_data = triples_maker(whole_data)
sample_data

[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')],
 [('inject', 'JJ', 'Method'), ('2', 'CD', 'Qty'), ('units', 'NNS', 'Form')],
 [('x', 'RB', 'FOR'),
  ('2', 'CD', 'Duration'),
  ('weeks', 'NNS', 'DurationUnit')],
 [('x', 'RB', 'FOR'),
  ('3', 'CD', 'Duration'),
  ('days', 'NNS', 'DurationUnit')],
 [('every', 'DT', 'EVERY'), ('day', 'NN', 'Period')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('3', 'CD', 'Period'),
  ('days', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('1', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('2', 'CD', 'PeriodMax'),
  ('months', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('2', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('weeks', 'NNS', 'PeriodUnit')],
 [('every', 'DT', 'EVERY'),
  ('4', 'CD', 'Period'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'PeriodMax'),
  ('

## Extract features

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag

In [8]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

## Split the data into training and testing set

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [9]:
# A function for extracting features in document
def get_features (doc):
    return [token_to_features (doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [triple[2] for triple in doc]

X = [get_features(doc) for doc in sample_data]
y = [get_labels(doc) for doc in sample_data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Train a CRF model

### Training the CRF model with the features extracted using the feature extractor method

In [10]:
# Train the CRF model

# Initialise the trainer
trainer = pycrfsuite.Trainer(algorithm='lbfgs')

# Submit training data to the trainer
for x, y in zip(X_train, y_train):
    trainer.append(x, y)

# Set the parameters of the model
trainer.set_params({
    'c1': 0.1,
    'c2': 0.1,
    'max_iterations': 100,
    'feature.possible_transitions': True
})

# Providing a file name as a parameter to the train function, such that the model will be saved to the file when training is finished
trainer.train('crf_model.crfsuite')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 1763
Seconds required: 0.003

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 568.143184
Feature norm: 1.000000
Error norm: 116.003715
Active features: 1749
Line search trials: 1
Line search step: 0.006435
Seconds required for this iteration: 0.002

***** Iteration #2 *****
Loss: 387.359975
Feature norm: 5.053745
Error norm: 145.752464
Active features: 1489
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.005

***** Iteration #3 *****
Loss: 282.295063
Feature norm: 5.958821
Error norm: 65.263181
Active features: 1497
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 

## Predict on test data

### Predicting the test data with the built model

### X_test has features extracted from tokenized sentence already

In [11]:
tagger = pycrfsuite.Tagger()
tagger.open('crf_model.crfsuite')
y_pred = [tagger.tag(x) for x in X_test]
y_pred

[['FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit'],
 ['EVERY', 'Period', 'PeriodUnit'],
 ['QID'],
 ['Method', 'Qty', 'Form', 'TID', 'DAILY'],
 ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'],
 ['EVERY', 'Period', 'PeriodUnit'],
 ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'],
 ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'],
 ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'],
 ['Method',
  'Qty',
  'Form',
  'Frequency',
  'PeriodUnit',
  'FOR',
  'Duration',
  'DurationUnit',
  'AT',
  'WHEN'],
 ['FOR', 'Duration', 'DurationUnit'],
 ['FOR', 'Duration', 'DurationUnit']]

## Do sample predictions

### Putting all the prediction logic inside a predict method

In [12]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig  A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    """
    # Load the trained CRF model
    tagger = pycrfsuite.Tagger()
    tagger.open('crf_model.crfsuite')

    # Tokenize the input prescription sentence and make triples out of it
    tokenized_sentence = word_tokenize(sig)
    tags = pos_tag([word for word in tokenized_sentence])
    sentence = [(input_word, tag[1], 'LABEL') for input_word, tag in zip(tokenized_sentence, tags)]

    # Extract features for each word in the sentence
    features = [token_to_features(sentence, i) for i in range(len(sentence))]

    # Predict labels using the CRF model
    predictions = [tagger.tag(features)]
    
    return f"{sig}\n{predictions}"

In [13]:
predictions = predict("take 2 tabs every 6 hours x 10 days")
print(predictions)

take 2 tabs every 6 hours x 10 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [14]:
predictions = predict("2 capsu for 10 day at bed")
print(predictions)

2 capsu for 10 day at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [15]:
predictions = predict("2 capsu for 10 days at bed")
print(predictions)

2 capsu for 10 days at bed
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [16]:
predictions = predict("5 days 2 tabs at bed")
print(predictions)

5 days 2 tabs at bed
[['Duration', 'DurationUnit', 'Qty', 'Form', 'AT', 'WHEN']]


In [17]:
predictions = predict("3 tabs qid x 10 weeks")
print(predictions)

3 tabs qid x 10 weeks
[['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [18]:
predictions = predict("x 30 days")
print(predictions)

x 30 days
[['FOR', 'Duration', 'DurationUnit']]


In [19]:
predictions = predict("x 20 months")
print(predictions)

x 20 months
[['FOR', 'Duration', 'DurationUnit']]


In [20]:
predictions = predict("take 2 tabs po tid for 10 days")
print(predictions)

take 2 tabs po tid for 10 days
[['Method', 'Qty', 'Form', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit']]


In [21]:
predictions = predict("take 2 capsules po every 6 hours")
print(predictions)

take 2 capsules po every 6 hours
[['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit']]


In [22]:
predictions = predict("inject 2 units pu tid")
print(predictions)

inject 2 units pu tid
[['Method', 'Qty', 'Form', 'PO', 'TID']]


In [23]:
predictions = predict("swallow 3 caps tid by mouth")
print(predictions)

swallow 3 caps tid by mouth
[['Method', 'Qty', 'Form', 'TID', 'BY', 'PO']]


In [24]:
predictions = predict("inject 3 units orally")
print(predictions)

inject 3 units orally
[['Method', 'Qty', 'Form', 'Frequency']]


In [25]:
predictions = predict("orally take 3 tabs tid")
print(predictions)

orally take 3 tabs tid
[['Method', 'Method', 'Qty', 'Form', 'TID']]


In [26]:
predictions = predict("by mouth take three caps")
print(predictions)

by mouth take three caps
[['BY', 'PO', 'Method', 'Qty', 'Form']]


In [27]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")
print(predictions)

take 3 tabs orally three times a day for 10 days at bedtime
[['Method', 'Qty', 'Form', 'Frequency', 'Qty', 'TIMES', 'Period', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [28]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")
print(predictions)

take 3 tabs orally bid for 10 days at bedtime
[['Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [29]:
predictions = predict("take 3 tabs bid orally at bed")
print(predictions)

take 3 tabs bid orally at bed
[['Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN']]


In [30]:
predictions = predict("take 10 capsules by mouth qid")
print(predictions)

take 10 capsules by mouth qid
[['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']]


In [31]:
predictions = predict("inject 10 units orally qid x 3 months")
print(predictions)

inject 10 units orally qid x 3 months
[['Method', 'Qty', 'Form', 'Frequency', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [32]:
predictions = predict("please take 2 tablets per day for a month in the morning and evening each day")
print(predictions)

please take 2 tablets per day for a month in the morning and evening each day
[['Method', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Period', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit', 'AND', 'EVERY', 'Period', 'PeriodUnit']]


In [33]:
predictions = predict("Amoxcicillin QID 30 tablets")
print(predictions)

Amoxcicillin QID 30 tablets
[['Method', 'Method', 'Qty', 'Form']]


In [34]:
predictions = predict("take 3 tabs TID for 90 days with food")
print(predictions)

take 3 tabs TID for 90 days with food
[['Method', 'Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'WITH', 'FOOD']]


In [35]:
predictions = predict("with food take 3 tablets per day for 90 days")
print(predictions)

with food take 3 tablets per day for 90 days
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [36]:
predictions = predict("with food take 3 tablets per week for 90 weeks")
print(predictions)

with food take 3 tablets per week for 90 weeks
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [37]:
predictions = predict("take 2-4 tabs")
print(predictions)

take 2-4 tabs
[['Method', 'Qty', 'Form']]


In [38]:
predictions = predict("take 2 to 4 tabs")
print(predictions)

take 2 to 4 tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [39]:
predictions = predict("take two to four tabs")
print(predictions)

take two to four tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [40]:
predictions = predict("take 2-4 tabs for 8 to 9 days")
print(predictions)

take 2-4 tabs for 8 to 9 days
[['Method', 'Qty', 'Form', 'FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']]


In [41]:
predictions = predict("take 20 tabs every 6 to 8 days")
print(predictions)

take 20 tabs every 6 to 8 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [42]:
predictions = predict("take 2 tabs every 4 to 6 days")
print(predictions)

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [43]:
predictions = predict("take 2 tabs every 2 to 10 weeks")
print(predictions)

take 2 tabs every 2 to 10 weeks
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'Duration', 'DurationUnit']]


In [44]:
predictions = predict("take 2 tabs every 4 to 6 days")
print(predictions)

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [45]:
predictions = predict("take 2 tabs every 2 to 10 months")
print(predictions)

take 2 tabs every 2 to 10 months
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [46]:
predictions = predict("every 60 mins")
print(predictions)

every 60 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [47]:
predictions = predict("every 10 mins")
print(predictions)

every 10 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [48]:
predictions = predict("every two to four months")
print(predictions)

every two to four months
[['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [49]:
predictions = predict("take 2 tabs every 3 to 4 days")
print(predictions)

take 2 tabs every 3 to 4 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [50]:
predictions = predict("every 3 to 4 days take 20 tabs")
print(predictions)

every 3 to 4 days take 20 tabs
[['EVERY', 'Period', 'TO', 'Duration', 'DurationUnit', 'Method', 'Qty', 'Form']]


In [51]:
predictions = predict("once in every 3 days take 3 tabs")
print(predictions)

once in every 3 days take 3 tabs
[['Frequency', 'Period', 'EVERY', 'Period', 'PeriodUnit', 'Method', 'Qty', 'Form']]


In [52]:
predictions = predict("take 3 tabs once in every 3 days")
print(predictions)

take 3 tabs once in every 3 days
[['Method', 'Qty', 'Form', 'Frequency', 'Period', 'EVERY', 'Period', 'PeriodUnit']]


In [53]:
predictions = predict("orally take 20 tabs every 4-6 weeks")
print(predictions)

orally take 20 tabs every 4-6 weeks
[['Method', 'Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]


In [54]:
predictions = predict("10 tabs x 2 days")
print(predictions)

10 tabs x 2 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [55]:
predictions = predict("3 capsule x 15 days")
print(predictions)

3 capsule x 15 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [56]:
predictions = predict("10 tabs")
print(predictions)

10 tabs
[['Qty', 'Form']]
